# Run First

In [0]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My\ Drive/Code/CNN

installed_packages = !pip list

for x in ['pycuda', 'scipy', 'hiddenlayer']:
  if x not in installed_packages:
    print(x)
    !pip install $x

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/Code/CNN
pycuda
     |████████████████████████████████| 1.6MB 1.4MB/s 
     |████████████████████████████████| 61kB 24.9MB/s 
     |████████████████████████████████| 471kB 36.7MB/s 
  Created wheel for pycuda: filename=pycuda-2019.1.1-cp36-cp36m-linux_x86_64.whl size=4544648 sha256=780802f4daa2fbda09136224ece3ef51748f749c1ff1504a51822370e20f4a77
  Stored in directory: /root/.cache/pip/wheels/31/f5/72/73296e7845a1ddd8769ef5baa6a8bb05bbd8

# Import Packages, Load Dataset, Define Transforms

In [0]:
ModelName = 'ConvNet' # [ConvNet] [GoogleNet]
img_dir = './data' # Specificy path to CIFAR-10 dataset and set download yes/no flag
import os, sys
# Check current directory isn't root
if os.getcwd() == '/content':
  %cd /content/drive/My\ Drive/Code/CNN

import hiddenlayer as hl  
import pickle
from torchsummary import summary
from timeit import default_timer as timer
from datetime import timedelta
import numpy as np
import scipy.io as sio
import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lrs
import torchvision
import torchvision.transforms as transforms
from torch.utils import data
import torch.nn as nn
import torch.nn.functional as F

import pycuda.driver as cuda
cuda.init()
## Get Id of default device
torch.cuda.current_device()
cuda.Device(0).name()
print(cuda.Device(0).name())

############## CONFIGURATION ##############
NUM_WORKERS = 8
#### Adaptive LR specific parameters
LR_PATIENCE = 5 # This is the number of epochs to observe no change in before change LR
LR_FACTOR = 0.75  # factor by which to reduce the LR newLR = LR*LF_FACTOR
BATCH_SIZE = 128
train_iterations = int((50000/BATCH_SIZE)) 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# DEFINING TRANSFORM TO APPLY TO THE IMAGES
train_transform = transforms.Compose([
   transforms.RandomCrop(32, padding=4),
   transforms.RandomHorizontalFlip(),
   transforms.Resize(32),
	 transforms.ToTensor(),
	 transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

test_transform = transforms.Compose([
   transforms.Resize(32),
	 transforms.ToTensor(),
	 transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

########################################################################
# 1. LOAD AND NORMALIZE CIFAR10 DATASET
########################################################################
# Specify training and testing sets
trainset = torchvision.datasets.CIFAR10(root = img_dir, train = True, download = False, transform = train_transform)
testset = torchvision.datasets.CIFAR10(root = img_dir, train = False, download = False, transform = test_transform)
# Shuffle the training set only
trainloader = torch.utils.data.DataLoader(trainset, batch_size = BATCH_SIZE, shuffle = True, num_workers = NUM_WORKERS)
testloader = torch.utils.data.DataLoader(testset, batch_size = BATCH_SIZE, shuffle = False, num_workers = NUM_WORKERS)

def unpickle(file):
	with open(file, 'rb') as fo:
		meta_dict_b = pickle.load(fo, encoding = 'bytes')
		meta_dict = [x.decode('UTF-8') for x in meta_dict_b[b'label_names']]
	return meta_dict

def get_lr(optimizer):
	for param_group in optimizer.param_groups:
		return param_group['lr']    

Tesla T4


# Define Both Models

In [0]:
########################################################################
# 2. DEFINE YOUR CONVOLUTIONAL NEURAL NETWORK
########################################################################  
class Inception(nn.Module):
    def __init__(self, in_planes, kernel_1_x, kernel_3_in, kernel_3_x, kernel_5_in, kernel_5_x, pool_planes):
        super(Inception, self).__init__()
        # 1x1 conv branch
        self.b1 = nn.Sequential(
            nn.Conv2d(in_planes, kernel_1_x, kernel_size=1),
            nn.BatchNorm2d(kernel_1_x),
            nn.ReLU(True),
        )

        # 1x1 conv -> 3x3 conv branch
        self.b2 = nn.Sequential(
            nn.Conv2d(in_planes, kernel_3_in, kernel_size=1),
            nn.BatchNorm2d(kernel_3_in),
            nn.ReLU(True),
            nn.Conv2d(kernel_3_in, kernel_3_x, kernel_size=3, padding=1),
            nn.BatchNorm2d(kernel_3_x),
            nn.ReLU(True),
        )

        # 1x1 conv -> 5x5 conv branch
        self.b3 = nn.Sequential(
            nn.Conv2d(in_planes, kernel_5_in, kernel_size=1),
            nn.BatchNorm2d(kernel_5_in),
            nn.ReLU(True),
            nn.Conv2d(kernel_5_in, kernel_5_x, kernel_size=3, padding=1),
            nn.BatchNorm2d(kernel_5_x),
            nn.ReLU(True),
            nn.Conv2d(kernel_5_x, kernel_5_x, kernel_size=3, padding=1),
            nn.BatchNorm2d(kernel_5_x),
            nn.ReLU(True),
        )

        # 3x3 pool -> 1x1 conv branch
        self.b4 = nn.Sequential(
            nn.MaxPool2d(3, stride=1, padding=1),
            nn.Conv2d(in_planes, pool_planes, kernel_size=1),
            nn.BatchNorm2d(pool_planes),
            nn.ReLU(True),
        )

    def forward(self, x):
        y1 = self.b1(x)
        y2 = self.b2(x)
        y3 = self.b3(x)
        y4 = self.b4(x)
        return torch.cat([y1,y2,y3,y4], 1)


class GoogLeNet(nn.Module):
    def __init__(self):
        super(GoogLeNet, self).__init__()
        self.pre_layers = nn.Sequential(
            nn.Conv2d(3, 192, kernel_size=3, padding=1),
            nn.BatchNorm2d(192),
            nn.ReLU(True),
        )

        self.a3 = Inception(192,  64,  96, 128, 16, 32, 32)
        self.b3 = Inception(256, 128, 128, 192, 32, 96, 64)

        self.max_pool = nn.MaxPool2d(3, stride=2, padding=1)

        self.a4 = Inception(480, 192,  96, 208, 16,  48,  64)
        self.b4 = Inception(512, 160, 112, 224, 24,  64,  64)
        self.c4 = Inception(512, 128, 128, 256, 24,  64,  64)
        self.d4 = Inception(512, 112, 144, 288, 32,  64,  64)
        self.e4 = Inception(528, 256, 160, 320, 32, 128, 128)

        self.a5 = Inception(832, 256, 160, 320, 32, 128, 128)
        self.b5 = Inception(832, 384, 192, 384, 48, 128, 128)

        self.avgpool = nn.AvgPool2d(8, stride=1)
        self.linear = nn.Linear(1024, 10)

    def forward(self, x):
        x = self.pre_layers(x)
        x = self.a3(x)
        x = self.b3(x)
        x = self.max_pool(x)
        x = self.a4(x)
        x = self.b4(x)
        x = self.c4(x)
        x = self.d4(x)
        x = self.e4(x)
        x = self.max_pool(x)
        x = self.a5(x)
        x = self.b5(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.linear(x)
        return x

class ConvNet(nn.Module):
	def __init__(self, init_weights = False):
		super(ConvNet, self).__init__()
		
		self.conv1 = nn.Conv2d(3, WEIGHTS, 3)
		self.bnorm1 = nn.BatchNorm2d(WEIGHTS)
		self.conv2 = nn.Conv2d(WEIGHTS, WEIGHTS, 3)
		self.bnorm2 = nn.BatchNorm2d(WEIGHTS)
		self.drop_out1 = nn.Dropout(p = 0.2)
		
		self.conv3 = nn.Conv2d(WEIGHTS, WEIGHTS * 2, 3)
		self.bnorm3 = nn.BatchNorm2d(WEIGHTS * 2)
		self.conv4 = nn.Conv2d(WEIGHTS * 2, WEIGHTS * 2, 3)
		self.bnorm4 = nn.BatchNorm2d(WEIGHTS * 2)
		self.drop_out2 = nn.Dropout(p = 0.3)
		
		self.conv5 = nn.Conv2d(WEIGHTS * 2, WEIGHTS * 4, 3)
		self.bnorm5 = nn.BatchNorm2d(WEIGHTS * 4)
		self.conv6 = nn.Conv2d(WEIGHTS * 4, WEIGHTS * 4, 3)
		self.bnorm6 = nn.BatchNorm2d(WEIGHTS * 4)
		self.drop_out3 = nn.Dropout(p = 0.4)
		
		self.fc1 = nn.Linear((WEIGHTS * 4) * 4 * 4, 10)
		self.pool = nn.MaxPool2d(2, 2)  # 2x2 window with stride = 2
		self.softmax = nn.Softmax()
		
		if init_weights:
			self._initialize_weights()
	
	def forward(self, x):
		# Pad following pooling in order to maintaing 32x32 size
		pad1 = [1, 1, 1, 1]  # use below padding for alternate kernel sizes in subsequent layers--e.g. 3-4
		
		x = F.pad(self.bnorm1((F.relu(self.conv1(x)))), pad1, 'constant', 0)
		x = F.pad(self.bnorm2(F.relu(self.conv2(x))), pad1, 'constant', 0)
		x = self.drop_out1(self.pool(x))
		
		x = F.pad(self.bnorm3((F.relu(self.conv3(x)))), pad1, 'constant', 0)
		x = F.pad(self.bnorm4(F.relu(self.conv4(x))), pad1, 'constant', 0)
		x = self.drop_out2(self.pool(x))
		
		x = F.pad(self.bnorm5((F.relu(self.conv5(x)))), pad1, 'constant', 0)
		x = F.pad(self.bnorm6(F.relu(self.conv6(x))), pad1, 'constant', 0)
		x = self.drop_out3(self.pool(x))
		
		x = x.view(-1, (WEIGHTS * 4) * 4 * 4)
		x = self.fc1(x)
#		x = self.softmax(x)
		return x
	
	def _initialize_weights(self):
		for m in self.modules():
			if isinstance(m, nn.Conv2d):
				nn.init.kaiming_normal_(m.weight, mode = 'fan_out', nonlinearity = 'relu')
				if m.bias is not None:
					nn.init.constant_(m.bias, 0)
			elif isinstance(m, nn.BatchNorm2d):
				nn.init.constant_(m.weight, 1)
				nn.init.constant_(m.bias, 0)
			elif isinstance(m, nn.Linear):
				nn.init.normal_(m.weight, 0, 0.01)
				nn.init.constant_(m.bias, 0)  

# main()

In [0]:
import hiddenlayer

def main():
	########################################################################
	# 3. DEFINE A LOSS FUNCTION AND OPTIMIZER
	########################################################################
	criterion = nn.CrossEntropyLoss()
	optimizer = optim.SGD(net.parameters(), lr = LR, momentum = 0.9, weight_decay = 0.0001)
	 # Adaptive learning rate--if accuracy (over 2 epochs) doesn't increase then reduce it by 10%
	scheduler = lrs.ReduceLROnPlateau(optimizer, 'max',
	                                   factor = LR_FACTOR,
	                                   patience = LR_PATIENCE,
	                                   verbose = True,
	                                   threshold = 1)
	# scheduler = CyclicLR(optimizer, base_lr = 0.0001, max_lr = 0.1, step_size = half_cycle)  
	########################################################################
	# 4. TRAIN THE NETWORK
	########################################################################
	# Get class labels
	meta = img_dir + '/cifar-10-batches-py/batches.meta'
	classes = unpickle(meta)
	# Print model hyper-parameters and hidden parameters
	print('=' * 100)
	print('Classes: [%s]' % ', '.join(map(str, classes)))
	print('Number of Filters: ' + str(WEIGHTS))
	print('Batchsize = ' + str(BATCH_SIZE))
	print('Number of Batches = ' + str(train_iterations))
	summary(net, (3, 32, 32))
	print('=' * 100)
	test_accuracy = []
	train_accuracy = []
	train_loss = []
	history2 = hl.History()
	canvas2 = hl.Canvas() 
	step = (0,0)
    
	for epoch in range(1, EPOCH_NUM, 1):  # loop over the dataset multiple times
		# scheduler.batch_step()  # uncomment for CyclicLR
		print('Beginning Epoch ' + str(epoch) + ' with Learning Rate = ' + str(round(get_lr(optimizer), 5)))
		running_loss = 0.0
		test_min_acc = 0
		total = 0
		correct = 0
		for i, Data in enumerate(trainloader, 0):
			step = (epoch, i)
      # get the inputs
			inputs, labels = Data
			# zero the parameter gradients
			optimizer.zero_grad()
			# forward + backward + optimize
			outputs = net(inputs.to(device)).to(device)
			loss = criterion(outputs, labels.to(device))
			loss.backward()
			optimizer.step()
			# print statistics
			running_loss += loss.item()
			_, predicted = torch.max(outputs.data, 1)
			# Accuracy of given batch
			total += labels.size(0)
			correct += (predicted == labels.to(device)).sum().item()
			train_loss.append(running_loss / 20)
			train_accuracy.append(100.0 * correct / total)
			accuracy = 100.0 * correct / total
			if i % 20 == 19:  # print every 20 mini-batches
				print('Train: [%d, %5d] Loss: %.3f Acc: %.3f' % (epoch, i + 1, running_loss / 20, accuracy))
				running_loss = 0.0
			
		# TEST LEARNT MODEL ON TEST-SET
		correct = 0
		total = 0
		with torch.no_grad():
			for Data in testloader:
				images, labels = Data
				outputs = net(images.to(device))
				_, predicted = torch.max(outputs.data, 1)
				total += labels.to(device).size(0)
				correct += (predicted == labels.to(device)).sum().item()
		test_accuracy.append(100.0 * correct / total)
		test_ep_acc = test_accuracy[-1]
		# See if LR needs to be changed--used by ReduceLROnPlateau above
		scheduler.step(test_ep_acc)
		
		test_acc_str = '[ Epoch ' + str(epoch) + ' Test Accuracy = ' + str(round(test_ep_acc, 3)) + ' % ]'
		pad = 50 - int(round(float(len(test_acc_str) / 2)))
		print('=' * pad + test_acc_str + '=' * pad)
		# SAVE BEST MODEL
		if test_min_acc < test_ep_acc:
			test_min_acc = test_ep_acc
			torch.save(net, MODEL_SAVE_PATH + '/my_best_model.pth')
	
	np.save('test_accuracy.npy', test_accuracy);
	sio.savemat('test_accuracy.mat', mdict = {'test_accuracy': test_accuracy})
	np.save('train_accuracy.npy', train_accuracy);
	sio.savemat('train_accuracy.mat', mdict = {'train_accuracy': train_accuracy})
	np.save('train_loss.npy', train_loss);
	sio.savemat('train_loss.mat', mdict = {'train_loss': train_loss})
	
	print('Finished Training')  

# Define Hyper-Parameters & Execute

In [0]:
ModelName = 'GoogleNet'

######################################################
################## HYPER-PARAMETERS ##################
######################################################
WEIGHTS = 32
EPOCH_NUM = 15
LR = 0.1 # NOTE: LR is adaptive

######################################################
################ CREATE MODEL SUB-DIR ################
######################################################
img_dir = './data' # Specificy path to CIFAR-10 dataset and set download yes/no flag
MODEL_SAVE_PATH = './Output/'+ ModelName
cwd_path = os.getcwd()
if cwd_path == '/content': # Check current directory isn't root
    %cd /content/drive/My\ Drive/Code/CNN
mkdir_var = str(cwd_path +  '/Output/' + ModelName + '/Models').replace(" ", "\ ")
!mkdir -p $mkdir_var
# mkdir_var = str(cwd_path +  '/data/ImageNet/').replace(" ", "\ ")
# !mkdir -p $mkdir_var
print(MODEL_SAVE_PATH)



######################################################
##################### EXECUTE ########################
######################################################
if __name__ == "__main__":
    if ModelName == 'GoogleNet': net = GoogLeNet().to(device)
    else: net = ConvNet().to(device)
    start = timer()
    main()
    end = timer()
    execution_time = timedelta(seconds=end-start)
    print(execution_time)
    file = open(MODEL_SAVE_PATH + '/ExecutionTime.txt','w')
    file.writelines(str(execution_time)) 
    file.close() 


./Output/GoogleNet
Classes: [airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck]
Number of Filters: 32
Batchsize = 128
Number of Batches = 390
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 192, 32, 32]           5,376
       BatchNorm2d-2          [-1, 192, 32, 32]             384
              ReLU-3          [-1, 192, 32, 32]               0
            Conv2d-4           [-1, 64, 32, 32]          12,352
       BatchNorm2d-5           [-1, 64, 32, 32]             128
              ReLU-6           [-1, 64, 32, 32]               0
            Conv2d-7           [-1, 96, 32, 32]          18,528
       BatchNorm2d-8           [-1, 96, 32, 32]             192
              ReLU-9           [-1, 96, 32, 32]               0
           Conv2d-10          [-1, 128, 32, 32]         110,720
      BatchNorm2d-11          [-1, 128, 32, 32]             256
      

KeyboardInterrupt: ignored